In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from urllib.request import urlopen
from bs4 import  BeautifulSoup
import pandas as pd
from urllib.request import urlretrieve
import os
import requests

# 美麗版 https://www.everydayhealth.com.tw/category/48/index/1   有35頁
# 爬標題 URL div class grid calc-3
# 文章內容 https://www.everydayhealth.com.tw/article/16393
# 標題 日期 內容
c = ["title","URL","Published date","content"]
df= pd.DataFrame(columns=c)
for p in range(1,35+1):
	print(f"第{p}頁")
	try:
		url="https://www.everydayhealth.com.tw/category/48/index/"+str(p)
		print(url)
		response = urlopen(url)
		html = BeautifulSoup(response)
	except:
		print("跳過")
		sleep(3)

	article_l=html.find_all("div", class_="article-grid clear")
	# article_l 美麗最新文章，美麗最熱門文章
	for article_o in article_l:
		article=article_o.find_all("div", class_="grid calc-3")
		# 每一篇文章(美麗最新文章，美麗最熱門文章)
		for i in article:
			detail_o=i.find("a",class_="detail")
			detail_url=detail_o["href"]
			# 單一文章網址後綴

			url="https://www.everydayhealth.com.tw/"+str(detail_url)
			response = urlopen(url)
			html = BeautifulSoup(response)

			# 標題
			title = html.find("h1", itemprop="headline").text
			print(title)
			# 時間 span itemprop datePublished 
			pbl_time = html.find("span", itemprop="datePublished").text
			print(pbl_time)

			if html.find("ul", class_="pagination"):
				next_page = html.find("ul", class_="pagination")
				next_page_url_l = next_page.find_all("a")
				article_content = ""
				for k in range(len(next_page_url_l)):
					# 如果文章有多頁 一頁一頁拿內容 ul class="pagination"
					# ttps://www.everydayhealth.com.tw/article/22237
					# https://www.everydayhealth.com.tw/article/22237/2
					if k == 0 or k == len(next_page_url_l)-1:
						continue
					else:
						next_page_url = next_page_url_l[k]["href"]
						response = urlopen(next_page_url)
						html = BeautifulSoup(response)
						article_content += html.find("div", id="article_page").text
						print(article_content)
			else:
				# 文章只有一頁 直接拿內容
				# 文章內容 div id="article_page"
				article_content = html.find("div", id="article_page").text
				print(article_content)
			data = [title, url, pbl_time, article_content]
			s = pd.Series(data, index=c)
			df = df.append(s, ignore_index=True)
df.to_csv("beauty_health_lee_20190809.csv", encoding="utf-8", index=False)


In [ ]:
import pymongo
import pandas as pd

df = pd.read_csv("beauty_health_lee_20190809.csv")

def dataframe_to_mongo(select_df):
	client = pymongo.MongoClient("mongodb://192.168.71.128:27017/")
    # docker run --name mongo4 -v $(pwd)/mongodb_data:/data/db -d -p 27017:27017 --rm mongo:4.1
    # 透過VMware IP連進docker_mongodb
    database = client["db102"]  # Database Name
    collection = database["crawler_things"]   # Collection Name

    records = select_df.to_dict('records') # 參數 record 代表把列轉成個別物件
    collection.insert_many(records)

if __name__ == "__main__":
    dataframe_to_mongo(df)